In [ ]:
def preprocess(dataset_url, map_url, _pos=False, _prev=False, _can=False, _sparse=False, cnn_layers=None):
    df = pd.read_csv(dataset_url)
    with open(map_url, 'r') as f:
        map = json.load(f)

    X_all = []
    if _pos:
        X = df[['fet_x', 'fet_y', 'tar_x', 'tar_y']]
        X.fet_x /= 27
        X.fet_y /= 31
        X.tar_x /= 27
        X.tar_y /= 31
        X_all.append(X)
    if _prev:
        X = pd.get_dummies(df.prev_dir).iloc[:, :-1]
        X_all.append(X)
    if _can:
        X = []
        for i in np.array([df.fet_x, df.fet_y, df.prev_dir]).T:
            possible_dirs = [
                int([i[0], i[1]-1] in map),
                int([i[0]-1, i[1]] in map),
                int([i[0], i[1]+1] in map),
                int([i[0]+1, i[1]] in map),
            ]
            if i[2] != 4:
                possible_dirs[(i[2]+2) % 4] = 0
            X.append(possible_dirs)
        X = pd.DataFrame(X, columns=['can_u', 'can_l', 'can_d', 'can_r'])
        X_all.append(X)

    if _sparse:
        sparse_matrix = []
        for fet_x, fet_y, tar_x, tar_y in zip(df.fet_x, df.fet_y, df.tar_x, df.tar_y):
            map_grid = []
            for i in range(1, 29):
                row = []
                for j in range(1, 32):
                    if [i, j] not in map:
                        row.append([1, 0, 0, 0])
                    elif [i, j] == [fet_x, fet_y]:
                        row.append([0, 0, 1, 0])
                    elif [i, j] == [tar_x, tar_y]:
                        row.append([0, 0, 0, 1])
                    else:
                        row.append([0, 1, 0, 0])
                map_grid.append(row)

            sparse_matrix.append(map_grid)
        sparse_matrix = np.array(sparse_matrix)
        global sm
        sm = sparse_matrix
        model_cnn = models.Sequential([
            layers.InputLayer(input_shape=(*sparse_matrix.shape[1:],)),
            *cnn_layers,
            layers.Flatten()
        ])
        X = model_cnn(sparse_matrix)
        X_all.append(X)

    
    y = df.pop('dir')
    return X_all, y

def plot_it(history):
    history = pd.DataFrame(history.history)
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    axes[0].plot(history['acc'], color='black')
    axes[0].plot(history['val_acc'], color='red')
    axes[0].legend(['acc', 'val_acc'])
    axes[0].set_ylabel("acc")
    axes[0].set_xlabel("epochs")
    axes[1].plot(history['loss'][len(history['loss'])//4:], color='black')
    axes[1].plot(history['val_loss'][len(history['loss'])//4:], color='red')
    axes[1].legend(['loss', 'val_loss'])
    axes[1].set_xlabel("epochs")
    axes[1].set_ylabel("loss")
    plt.show()

def print_matrics(history):
    vl = np.argmin(history.history['val_loss'])
    l = np.argmin(history.history['loss'])
    a = np.argmax(history.history['acc'])
    va = np.argmax(history.history['val_acc'])

    print(f'''
        --> accuray
        index:\t\t{a}
        acc:\t\t{history.history['acc'][a]}
        val_acc:\t{history.history['val_acc'][a]}
        loss:\t\t{history.history['loss'][a]}
        val_loss:\t{history.history['val_loss'][a]}
        --> validatoin accuracy
        index:\t\t{va}
        acc:\t\t{history.history['acc'][va]}
        val_acc:\t{history.history['val_acc'][va]}
        loss:\t\t{history.history['loss'][va]}
        val_loss:\t{history.history['val_loss'][va]}
        --> validation loss
        index:\t\t{vl}
        acc:\t\t{history.history['acc'][vl]}
        val_acc:\t{history.history['val_acc'][vl]}
        loss:\t\t{history.history['loss'][vl]}
        val_loss:\t{history.history['val_loss'][vl]}
        --> loss
        index:\t\t{l}
        acc:\t\t{history.history['acc'][l]}
        val_acc:\t{history.history['val_acc'][l]}
        loss:\t\t{history.history['loss'][l]}
        val_loss:\t{history.history['val_loss'][l]}
    ''')

def evaluate_model(model_name):
    with open(model_name + '_model.json', 'r') as f:
        content = f.read()
        loaded_model_acc = models.model_from_json(content)
        loaded_model_val_acc = models.model_from_json(content)
        loaded_model_val_loss = models.model_from_json(content)

    accuracies = []

    print('acc')
    loaded_model_acc.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])
    loaded_model_acc.load_weights(model_name + '_weights_acc.h5')
    loss, acc = loaded_model_acc.evaluate(X, y)
    accuracies.append(acc)

    print('val_acc')
    loaded_model_val_acc.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])
    loaded_model_val_acc.load_weights(model_name + '_weights_val_acc.h5')
    loss, acc = loaded_model_val_acc.evaluate(X, y)
    accuracies.append(acc)

    print('val_loss')
    loaded_model_val_acc.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])
    loaded_model_val_acc.load_weights(model_name + '_weights_val_loss.h5')
    loss, acc = loaded_model_val_acc.evaluate(X, y)
    accuracies.append(acc)

    print("\nbest: " + ['acc', 'val_acc', 'val_loss'][accuracies.index(max(accuracies))])
    print(max(accuracies))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import models, layers, callbacks, optimizers
import tensorflow as tf
import pandas as pd
import json, joblib
from google.colab import files

model_name = 'manual'

# Preprocessing

In [ ]:
X_all, y = preprocess(
	model_name + '_dataset.csv',
	'mapping.json',
	_pos=True, _prev=False, _can=True, _sparse=False,
	cnn_layers = [
	layers.Conv2D(32, 2, padding='same', activation='relu'),
	layers.Conv2D(64, 2, padding='same', activation='relu'),
	]
)
X = np.column_stack(X_all)
X.shape, y.shape

# Using Tensorflow

In [ ]:
model = models.Sequential([
    layers.Dense(units=400, activation='relu', input_shape=(X.shape[1],)),
    layers.Dense(units=700, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(units=80, activation='relu'),
    layers.Dense(units=4, activation='sigmoid'),
])
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['acc'])
early_callback = callbacks.EarlyStopping(patience=300, monitor='acc', mode='max')
save_callback_acc = callbacks.ModelCheckpoint(
	filepath= model_name + '_weights_acc.h5',
	save_weights_only=True,
	frequency='epoch',
	save_best_only=True,
	monitor='acc',
	mode='max',
)
save_callback_val_acc = callbacks.ModelCheckpoint(
	filepath= model_name + '_weights_val_acc.h5',
	save_weights_only=True,
	frequency='epoch',
	save_best_only=True,
	monitor='val_acc',
	mode='max',
)
save_callback_val_loss = callbacks.ModelCheckpoint(
	filepath= model_name + '_weights_val_loss.h5',
	save_weights_only=True,
	frequency='epoch',
	save_best_only=True,
	monitor='val_loss',
	mode='min',
)
with open(model_name + '_model.json', 'w') as f:
    f.write(model.to_json())
history = model.fit(X, y, epochs=2000, validation_split=0.25, verbose=2, callbacks=[early_callback, save_callback_acc, save_callback_val_acc, save_callback_val_loss])
plot_it(history)
print_matrics(history)

# Efficiency of Each Model

In [ ]:
evaluate_model(model_name)

# Using Sklearn

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
def warn(*args, **kwargs): pass
warnings.warn = warn

sklearn_models = {
    'knn': KNeighborsClassifier(n_neighbors=25, weights='uniform', metric="minkowski", p=2),
    'scv': SVC(C = 10, kernel = "poly"),
    'log': LogisticRegression(),
    'logcv': LogisticRegressionCV(),
    'tree': DecisionTreeClassifier(criterion="entropy"),
    'forest': RandomForestClassifier(n_estimators=10, criterion="entropy"),
    'gaus': GaussianNB(),
}
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
sklearn_predictions = {}
sklearn_accuracies = {}
for key in sklearn_models:
    sklearn_models[key].fit(X_train, y_train)
    pred = sklearn_models[key].predict(X_train)
    sklearn_predictions[key] = pred
    sklearn_accuracies[key] = accuracy_score(y_train, pred)

print(sklearn_accuracies)